### Uses information from a submitter PF spreadsheet to add files to the appropriate place in indicated linked items
### Currently for ExperimentSets opfs but should be extend to Experiments, Pubs and possibly pages

#### Setup

- Provide a title and description to use for the opf section.
- indicate path to the workbook

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import get_key, digest_xlsx, reader

# get key from keypairs.json
my_key = get_key('andyprod')

# set title here
opf_type = 'supplementary'
opf_title = "Analysis results provided by the data submitters - the Diao Lab"
opf_desc = "The results were generated by the Diao lab using the code available at https://github.com/jianhong/hicar/releases/tag/2.0.0rc"

# location of excel processed file sheet
xcel_file = '/Users/andrew/Documents/work/4DN_Metadata/Diao_Yarui_lab/HiCAR_RNAseq_myoblast_diff/230830_hicar_processed_results_ajs_upd.xlsx'
xcel, sheets = digest_xlsx(xcel_file)
xcel_data = reader(xcel)

fieldnames = next(xcel_data)

# create a dict with dataset 2 processd files 
dset2opfs = {}
for row in xcel_data:
    if row[0].startswith('#'):
        continue
    data = dict(zip(fieldnames, row))
    data = {k: v for k, v in data.items() if v}
    file_alias = data.get('aliases')
    dataset = data.get('# linked_datasets')
    dset2opfs.setdefault(dataset, []).append(file_alias)
    
print(dset2opfs)

In [ ]:
# go through the dictionary and create a patch for the dataset 
# need to check and maintain other opfs

# if True do the action, if false just report
action = True

for ds, opfs in dset2opfs.items():
    opf_obj = {'type': opf_type, 'title': opf_title, 'files': opfs}
    if opf_desc:
        opf_obj['description'] = opf_desc
    
    dset = ff_utils.get_metadata(ds, my_key, add_on='frame=raw')
    curr_opfs = dset.get('other_processed_files', [])
    if curr_opfs:
        curr_titles = [i.get('title') for i in curr_opfs]
        if opf_title in curr_titles:
            print('ERROR: {} has been used as a title already for {} - NO GO!'.format(ds, opf_title))
            continue
    # in this specific case we want to insert the new one into
    # the first position of the list
    new_opf_grps = [opf_obj] + curr_opfs
    
    patch_data = {'other_processed_files': new_opf_grps}
    if action:
        res = ff_utils.patch_metadata(patch_data, dset['uuid'], key = my_key)
        print(res)
    else:
        print("\n")
        print("{}\t{}".format(dset.get('uuid'), dset.get('accession')))
        print(opf_obj)
        print("\n\n")
        print(patch_data)
 
    
